In [7]:
%pip install google-generativeai
%pip install openai
import google.generativeai as genai
import openai
import getpass


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [8]:
print("Enter the model you want to use: openai or gemini")
model_name = input()
print("You selected: ", model_name)
print("Enter your Gemini/openai API key: Please Note this is a demo version, only Gemini and openai is currently supported. Input is hidden")
API_KEY = getpass.getpass()
#model = genai.GenerativeModel("gemini-1.5-flash")
#response = model.generate_content("Write a story about a magic backpack")
#print(response.text)

Enter the model you want to use: openai or gemini
You selected:  openai
Enter your Gemini/openai API key: Please Note this is a demo version, only Gemini and openai is currently supported. Input is hidden


In [9]:
def gemini_chatbot_comm(user_input, history=[]):
    genai.configure(api_key=API_KEY)
    model = genai.GenerativeModel("gemini-1.5-flash")
    chat = model.start_chat(
    history = history
    )
    generation_config = {
        "temperature": 0.1  # Adjust temperature for more deterministic (lower) or more creative (higher) responses
    }
    response = chat.send_message(user_input, generation_config=generation_config)
    history.append({
        "role": "user",
        "parts": [{"text": user_input}]
    })
    history.append({
        "role": "model",
        "parts": [{"text": response.text}]
    })
    return response.text, history

In [10]:
def openai_chatbot_comm(user_input, history=[]):
    openai.api_key = API_KEY
    history.append({
        "role": "user",
        'content': user_input
    })
    
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini", 
        temperature=0.1,
        messages=history
    )
    history.append({
        "role": "assistant",
        'content': response.choices[0].message.content
    })
    return response.choices[0].message.content, history



# response, history = openai_chatbot_comm("Hello")
# print(response)
    

In [11]:
def chatbot_comm(user_input, model, history=[]):
    if 'gemini' in model.lower():
        response, history = gemini_chatbot_comm(user_input, history)
        return response, history
    elif 'openai' in model.lower():
        response, history = openai_chatbot_comm(user_input, history)
        return response, history

    

# Moderation is not working due to exceeded limit. Will implement later once billing is sorted.
# def moderate_response(response):
#     url = "https://api.openai.com/v1/moderations"
#     from openai import OpenAI
#     client=OpenAI(api_key=api_key)
#     response = client.moderations.create(input=response)
#     return response.results[0].flagged

def collect_info(user_input, model, history=[]):
    sys_message2 = """
    Remember to ask the questions one by one and wait for the user's response before asking the next question.
    Keep asking them questions to understand their needs better. The Following data needs to be collected: 
    1. User's budget 
    2. GPU Intensity 
    3. Multitasking 
    4. Portability
    5. Processing Speed
    6. Display Quality
    The Values for the above parameters should only be Low, Medium, High. Only Budget can be a number in INR. 
    You need to infer the values from the user's input as shown in the example below:
    -- GPU Intensity : High
    -- Multitasking : Medium
    -- Portability : Low
    -- Processing Speed : High
    -- Display Quality : High
    -- Budget : 50000
    Keep asking questions until you have all the information you need. 
    in the response, do not include the above instructions or response.
    If 'Not Done', continue asking questions.
    if 'Done', display the collected information in the above format
    Mandatorily respond with 'Done' or 'Not Done' after the response.
    """
    combined_input = f"{sys_message2}\n\nUser: {user_input}"
    response, history = chatbot_comm(combined_input, model, history)
    print("bot:", response.split('\n')[0])
    print("-"*20)
    # user_input = input()
    # print("user:", user_input)
    while 'Not Done' in response:
        user_input = input()
        print("user:", user_input)
        response, history = chatbot_comm(user_input, model, history)
        print("bot:", response.split('\n')[0])
        print("-"*20)
        #print('test 1 ------------------------------')
    print(response.split('\n')[0])
    return response

def display_info(data, model):
    sys_message3 = f"""
    You are the most intelligent laptop recommendation assistant.
    The information collected from the user is as follows:
    {data}
    Use this information to recommend the best laptop for the user. Use your own training data to make recommendations. Its fine even if its not up-to-date.
    respond with the name of the laptop, its price, and and a small description of the laptop.
    Respond with three best recommendations.
    """
    response, history = chatbot_comm(sys_message3, model)
    print("bot:", response)
    print("-"*20)
    


def initialize_conversation(model):
    print("Welcome to the Laptop Recommendation Chatbot! Please Note this is a demo version. ")
    sys_message = """You are a helpful assistant that can help user choose the best laptop for their needs. 
    Greet the user and ask how you can help them.
    """
    #combined_input = f"{sys_message}\n\nUser: {user_input}"
    response, history = chatbot_comm(sys_message, model)
    print("Bot: ", response)
    user_input = input()
    print("You: ", user_input)
    #mod_resp = moderate_response(response)
    #print(mod_resp)
    # print("Bot: ", response)
    # user_input = input()
    # print("You: ", user_input)
    # response, history = chatbot_comm(user_input, history)
    # print("Bot: ", response)
    data = collect_info(user_input, model, history) 
    display_info(data, model)
    #print(history)


initialize_conversation(model_name)



Welcome to the Laptop Recommendation Chatbot! Please Note this is a demo version. 
Bot:  Hello! I'm here to help you find the perfect laptop for your needs. What specific requirements or preferences do you have in mind? Are you looking for something for gaming, work, school, or general use? Let me know how I can assist you!
You:  I am looking for a gaming laptop
bot: Great! Gaming laptops typically require a good budget and strong specifications. What is your budget in INR? 
--------------------
user: My Budget is 60000
bot: Thank you! Now, how intense do you expect your gaming experience to be? Would you say the GPU intensity is Low, Medium, or High? 
--------------------
user: I am planning on playing High FPS Games
bot: Got it! So we'll categorize the GPU intensity as High. Next, how do you plan to use the laptop in terms of multitasking? Would you say your multitasking needs are Low, Medium, or High? 
--------------------
user: I only plan on playing games
bot: Understood! We'll co